In [15]:
import h5py
import sys
import os
import csv
import math

In [5]:
f = h5py.File('input/3.hdf5', 'r')
# List of device groups
deviceList = list(f.keys())
print(deviceList)
# Device 1
device = f[deviceList[0]]
print(device)
# Position dataset of device 1
dataset = device['Position']
print(dataset)
# Dataset of sample 1 of device 1
sample1Dataset = dataset[:, 0, :]
sampleNum = 0
xSum = 0
ySum = 0
zSum = 0
for sample in sample1Dataset:
    sampleNum += 1
    xSum += sample[0]
    ySum += sample[1]
    zSum += sample[2]
print(xSum/sampleNum)
print(ySum/sampleNum)
print(zSum/sampleNum)

['Device_792888129_0']
<HDF5 group "/Device_792888129_0" (9 members)>
<HDF5 dataset "Position": shape (238, 1, 3), type "<f4">
173.57560345305114
-299.29685756138394
-35.91537079690885


In [6]:
csv_file = 'text.csv'
shape = dataset.shape
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    for i in range(shape[1]):
        writer.writerows(dataset[:,0,:])

In [7]:
class OutputDictionary:
    def __init__(self):
        self.data = {}
    
    def addFile(self, filename):
        self.data[filename] = []
    
    def addDeviceAvg(self, filename, deviceName, avgList):
        deviceDict = {deviceName: avgList}
        self.data[filename].append(deviceDict)
    
    def __str__(self):
        return str(self.data)

In [8]:
nest = OutputDictionary()
nest.addFile("1.hdf5")
nest.addDeviceAvg("1.hdf5", "lights", [23, 54, 65])
nest.addDeviceAvg("1.hdf5", "BAR", [232, 544, 625])
nest.addFile("2.hdf5")
nest.addDeviceAvg("2.hdf5", "arm", [33, 56, 67])
print(nest)

{'1.hdf5': [{'lights': [23, 54, 65]}, {'BAR': [232, 544, 625]}], '2.hdf5': [{'arm': [33, 56, 67]}]}


Max Distance Test

In [18]:
class Sample:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
sampleList = []

fileH = h5py.File('input/3.hdf5', 'r')
deviceList = list(fileH.keys())
# Loops through device groups in file
for device in deviceList:
    # Checks if device has Position dataset
    tempDevice = fileH[device]
    if 'Position' in tempDevice:
        positionDataset = tempDevice['Position']
        shape = positionDataset.shape
        # Loops through each sensor in the device
        sensorIndex = 1
        for i in range(shape[sensorIndex]):
            sensorDataset = positionDataset[:, i, :]
            for sample in sensorDataset:
                point = Sample(sample[0], sample[1], sample[2])
                sampleList.append(point)

def euclideanDistance(sample1, sample2):
    x1, y1, z1 = sample1.x, sample1.y, sample1.z
    x2, y2, z2 = sample2.x, sample2.y, sample2.z
    squaredDistance = (x2 - x1)**2 + (y2-y1)**2 + (z2-z1)**2
    return math.sqrt(squaredDistance)

In [23]:
def maxDistance(sampleList):
    n = len(sampleList)
    maxDistance = 0
 
    # Iterate over all possible pairs
    for i in range(n):
        for j in range(i + 1, n):
             
            # Update maxm
            maxDistance = max(maxDistance, euclideanDistance(sampleList[i], sampleList[j]))
    return maxDistance

print(maxDistance(sampleList))

0.9714033812329474
